# Mintel Database Company Name Disambiguation
This notebook attempts to standardize company names from the Mintel Product Database with a fuzzy matching strategy.

## Reading in and Checking Data

In [11]:
# import packages
import pandas as pd
import numpy as np
import re
import nltk
from fuzzywuzzy import fuzz
from sklearn import cluster

In [12]:
# read in company data
companies = pd.read_csv('/Users/ncarlson/Dropbox/Mintel Immigration Project/GNPD-2009-19.csv', encoding = "cp1252")
# print head of file
print(companies[1:15])

             Ultimate Company                    Company  \
1                          22                         22   
2                          30                         30   
3                          31                         31   
4                         899                        899   
5                         999                        999   
6                         999                        999   
7                        1701                       1701   
8                        1907                       1907   
9                        2376                       2376   
10                     232605                     232605   
11   Hong Guan Kamtai (China)   Hong Guan Kamtai (China)   
12           Êxito Naturavene           Êxito Naturavene   
13                     Ñaming                     Ñaming   
14                     Ñapi-ú                     Ñapi-ú   

                                      Company address Company county/State  \
1      40 Vu Xuan Thi

In [13]:
# number of unique companies
len(pd.unique(companies['Ultimate Company']))

197175

In [50]:
# smaller subset of data to test code
companies_sample = companies.sample(n=10000)

## Data Cleaning and Processing
Next, we'll clean and preprocess the company names.

In [15]:
# function for removing special characters
def clean_special_characters(txt):
    seps = [' ','`','~','!','@','#','$','%','^','&','*','(',')','_','-','+','=','{','[','}','}','|',':',';','<',',','>','.','?','/']
    default_sep = seps[0]
    
    for sep in seps[1:]:
            txt = txt.replace(sep, default_sep)
    re.sub(' +', ' ', txt)
    temp_list = [i.strip() for i in txt.split(default_sep)]
    temp_list = [i for i in temp_list if i]
    return ' '.join(temp_list)

In [16]:
# function for removing stopwords -- using nltk english default for now
from nltk.corpus import stopwords
print(stopwords.words('english'))
my_stopwords = stopwords.words('english')

def clean_stopwords(txt):
    temp_list = txt.split(' ')
    temp_list = [i for i in temp_list if i not in my_stopwords]
    return ' '.join(temp_list)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [30]:
# data cleaning function -- putting the previous two together 
def data_cleaning(data, nameCol='Ultimate Company'):
    data.dropna(subset=[nameCol], inplace=True)
    data = data.rename_axis('CompanyID').reset_index()
    data_clean = data.copy()
    data_clean['CompanyName_clean'] = data_clean[nameCol].apply(lambda x: x.lower())
    data_clean['CompanyName_clean'] = data_clean['CompanyName_clean'].apply(clean_special_characters)
    data_clean['CompanyName_clean'] = data_clean['CompanyName_clean'].apply(clean_stopwords)
    data_clean = data_clean.sort_values(by=['CompanyName_clean'])
    return(data_clean)

In [51]:
# run data cleaning function on our data
companies_clean = data_cleaning(companies_sample, nameCol='Ultimate Company')
print(len(companies_clean))
# remove empty company names
companies_clean = companies_clean[companies_clean['CompanyName_clean']!='']
print(len(companies_clean))

10000
9995


## Fuzzy Matching
Next, we compute distance using the FuzzyWuzzy library. It's intractable to compute pairwise distance between 200,000^2 matches so we'll group by first character(s) and compute within. We will test out different metrics -- e.g. Levenschtein ratio, partial ratio, etc. -- and different cutoffs to choose the optimal approach. See this guide for more details on the specific ratios: https://www.datacamp.com/community/tutorials/fuzzy-string-python

In [52]:
# fuzzy matching by group 
from fuzzywuzzy import fuzz


# Levenschtein ratio
all_main_name = pd.DataFrame(columns=['first_characters','names','alias','score'])
all_names = companies_clean['CompanyName_clean'].unique()
all_main_name['names'] = all_names
all_main_name['first_characters'] = all_main_name['names'].apply(lambda x: x[0])
all_sort_gp = all_main_name['first_characters'].unique()

for sortgp in all_sort_gp:
    this_gp = all_main_name.groupby(['first_characters']).get_group(sortgp)
    gp_start = this_gp.index.min()
    gp_end = this_gp.index.max()
    for i in range(gp_start,gp_end+1):
    
        # if self has not got alias, asign to be alias of itself
        if pd.isna(all_main_name['alias'].iloc[i]):
            all_main_name['alias'].iloc[i] = all_main_name['names'].iloc[i]
            all_main_name['score'].iloc[i] = 100
        
        # if the following has not got alias and fuzzy match, asign to be alias of this one
        for j in range(i+1,gp_end+1):
            if pd.isna(all_main_name['alias'].iloc[j]):
                fuzz_score = fuzz.ratio(all_main_name['names'].iloc[i],all_main_name['names'].iloc[j])
                if (fuzz_score > 85):
                    all_main_name['alias'].iloc[j] = all_main_name['alias'].iloc[i]
                    all_main_name['score'].iloc[j] = fuzz_score

all_main_name.to_csv('/Users/ncarlson/Dropbox/Mintel Immigration Project/Levenschtein.csv')

# Partial ratio
all_main_name = pd.DataFrame(columns=['first_characters','names','alias','score'])
all_names = companies_clean['CompanyName_clean'].unique()
all_main_name['names'] = all_names
all_main_name['first_characters'] = all_main_name['names'].apply(lambda x: x[0])
all_sort_gp = all_main_name['first_characters'].unique()

for sortgp in all_sort_gp:
    this_gp = all_main_name.groupby(['first_characters']).get_group(sortgp)
    gp_start = this_gp.index.min()
    gp_end = this_gp.index.max()
    for i in range(gp_start,gp_end+1):
    
        # if self has not got alias, asign to be alias of itself
        if pd.isna(all_main_name['alias'].iloc[i]):
            all_main_name['alias'].iloc[i] = all_main_name['names'].iloc[i]
            all_main_name['score'].iloc[i] = 100
        
        # if the following has not got alias and fuzzy match, asign to be alias of this one
        for j in range(i+1,gp_end+1):
            if pd.isna(all_main_name['alias'].iloc[j]):
                fuzz_score = fuzz.partial_ratio(all_main_name['names'].iloc[i],all_main_name['names'].iloc[j])
                if (fuzz_score > 85):
                    all_main_name['alias'].iloc[j] = all_main_name['alias'].iloc[i]
                    all_main_name['score'].iloc[j] = fuzz_score

all_main_name.to_csv('/Users/ncarlson/Dropbox/Mintel Immigration Project/Partial.csv')


# harmonic mean of token set ratio and partial ratio
all_main_name = pd.DataFrame(columns=['first_characters','names','alias','score'])
all_names = companies_clean['CompanyName_clean'].unique()
all_main_name['names'] = all_names
all_main_name['first_characters'] = all_main_name['names'].apply(lambda x: x[0])
all_sort_gp = all_main_name['first_characters'].unique()

for sortgp in all_sort_gp:
    this_gp = all_main_name.groupby(['first_characters']).get_group(sortgp)
    gp_start = this_gp.index.min()
    gp_end = this_gp.index.max()
    for i in range(gp_start,gp_end+1):
    
        # if self has not got alias, asign to be alias of itself
        if pd.isna(all_main_name['alias'].iloc[i]):
            all_main_name['alias'].iloc[i] = all_main_name['names'].iloc[i]
            all_main_name['score'].iloc[i] = 100
        
        # if the following has not got alias and fuzzy match, asign to be alias of this one
        for j in range(i+1,gp_end+1):
            if pd.isna(all_main_name['alias'].iloc[j]):
                s1 = fuzz.partial_ratio(all_main_name['names'].iloc[i],all_main_name['names'].iloc[j])
                s2 = fuzz.token_set_ratio(all_main_name['names'].iloc[i],all_main_name['names'].iloc[j])
                fuzz_score = 2*s1*s2 / (s1 + s2)
                if (fuzz_score > 85):
                    all_main_name['alias'].iloc[j] = all_main_name['alias'].iloc[i]
                    all_main_name['score'].iloc[j] = fuzz_score

all_main_name.to_csv('/Users/ncarlson/Dropbox/Mintel Immigration Project/Harmonic_Mean.csv')
